In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
from convection_param.HelperFuncs import plot_icon_tricolor
import os
import re
from matplotlib import colors

%load_ext autoreload
%autoreload 2

base_path_project = '/work/bd1179/b309215'
base_path_scratch = '/scratch/b/b309215'

In [2]:
orig_file = "f'{base_path_scratch}/HErZ-NARVALII/DATA/2016082400/dei4_NARVALII_2016082400_fg_DOM01_ML_0001.nc"
orig_ds = xr.open_dataset(orig_file)
z_ifc_approx_km = np.mean(orig_ds['z_ifc'].values, axis=1)/1e3
z_fl_approx_km = (z_ifc_approx_km[1:] + z_ifc_approx_km[:-1]) /2

In [3]:
files_lr = sorted([f.path for f in os.scandir('../../PreprocessPipeline/RhoFluctTest/') if not f.is_dir()])
files_hr = sorted([f.path for f in os.scandir('../../PreprocessPipeline/RhoFluctTestHr/') if not f.is_dir()])
print(len(files_lr), len(files_hr))

8 8


In [ ]:
z_ifc = test_ds.z_ifc.values
print(z_ifc.shape)
for i,layer in enumerate(z_ifc):
    print(i, ': ', layer.mean(), layer.max() - layer.min())

(76, 4887488)
0 :  30000.0 0.0
1 :  28795.037 0.0
2 :  27645.252 0.0
3 :  26548.064 0.0
4 :  25500.943 0.0
5 :  24500.998 0.0
6 :  23546.775 0.0
7 :  22636.08 0.0
8 :  21767.98 0.0
9 :  20940.08 0.0
10 :  20151.916 0.0
11 :  19401.344 0.0
12 :  18687.98 0.0
13 :  18010.094 0.0
14 :  17367.324 0.0
15 :  16758.322 0.0
16 :  16182.348 0.0
17 :  15639.202 13.616211
18 :  15126.996 16.658203
19 :  14645.633 20.110352
20 :  14194.305 23.969727
21 :  13794.493 27.979492
22 :  13394.732 32.631836
23 :  12994.987 38.02832
24 :  12595.291 44.28125
25 :  12195.635 51.5166
26 :  11796.028 59.881836
27 :  11396.494 69.543945
28 :  10997.019 80.68945
29 :  10597.633 93.53418
30 :  10198.334 108.32031
31 :  9799.141 125.322266
32 :  9400.066 144.84668
33 :  9003.462 167.10156
34 :  8616.745 191.90918
35 :  8239.684 219.43457
36 :  7872.1523 249.83496
37 :  7514.0034 283.2583
38 :  7165.05 319.8423
39 :  6825.207 360.3667
40 :  6494.3315 406.60986
41 :  6172.283 457.26758
42 :  5858.96 512.8779
43 :  

In [ ]:
test_ds = xr.open_dataset(f'{base_path_scratch}/HErZ-NARVALII_WGrid/DATA/2013123000/dei4_NARVALI_2013123000_fg_DOM01_ML_0000.nc')

fig, ax = plt.subplots(subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,8))

mapplot = plot_icon_tricolor(ax, test_ds.clon_bnds.values, test_ds.clat_bnds.values, test_ds.z_ifc.values[0], cmap='bwr')#, norm=colors.TwoSlopeNorm(0))

ax.coastlines()
ax.gridlines(draw_labels=True)
fig.colorbar(mapplot, orientation='horizontal', aspect=30, pad=0.1)

plt.show()

In [10]:
base_fig_path = 'Figures/rho_fluctuations'
vertical_layers = [0,40, 70, 74]

for file_lr, file_hr in list(zip(files_lr, files_hr))[:]:
    date = re.search('_(201\d\d\d\d\d00)_', file_hr).group(1)
    hr = re.search('_(00\d\d)_', file_hr).group(1)
    print(f'Date: {date}, hour: {hr}')
    fig_path = os.path.join(base_fig_path, date, hr)
    os.makedirs(fig_path, exist_ok=True)

    lr_ds = xr.open_dataset(file_lr)
    lr_ds = lr_ds.rename({'rho_e': 'rho'})
    hr_ds = xr.open_dataset(file_hr)
    
    for vlvl in vertical_layers:
        print(f'Vertical lvl: {vlvl}')
        rho_lr_lvl_vals = lr_ds.rho.values.squeeze()[vlvl]
        rho_hr_lvl_vals = hr_ds.rho.values.squeeze()[vlvl]
        
        current_cmap = plt.get_cmap(name='seismic').copy()
        current_cmap.set_bad(color='#bcb5bb')

        # Plot absolute deviations
        rho_fluct = (rho_hr_lvl_vals - rho_lr_lvl_vals)

        fig, ax = plt.subplots(subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,8))
        mapplot = plot_icon_tricolor(ax, lr_ds.clon_bnds.values, lr_ds.clat_bnds.values, rho_fluct, cmap=current_cmap, norm=colors.TwoSlopeNorm(0))

        ax.coastlines()
        ax.gridlines(draw_labels=True)
        fig.colorbar(mapplot, orientation='horizontal', aspect=30, pad=0.1, label=r'$\rho - \bar\rho$ / kg m-3')
        fig.suptitle(fr'$\rho^\prime$ on vertical level {vlvl} ({z_fl_approx_km[vlvl]} km)')
        fig.savefig(os.path.join(fig_path, f'rho_fluct_abs_lvl{vlvl}.jpg'), dpi=150)
        plt.clf()

        # Plot relative deviations
        rho_fluct = rho_fluct / rho_lr_lvl_vals
        fig, ax = plt.subplots(subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,8))
        mapplot = plot_icon_tricolor(ax, lr_ds.clon_bnds.values, lr_ds.clat_bnds.values, rho_fluct, cmap=current_cmap, norm=colors.TwoSlopeNorm(0))

        ax.coastlines()
        ax.gridlines(draw_labels=True)
        fig.colorbar(mapplot, orientation='horizontal', aspect=30, pad=0.1, label=r'$(\rho - \bar\rho)/\bar\rho$')
        fig.suptitle(fr'$\rho^\prime / \bar\rho$ on vertical level {vlvl} ({z_fl_approx_km[vlvl]} km)')
        fig.savefig(os.path.join(fig_path, f'rho_fluct_rel_lvl{vlvl}.jpg'), dpi=150)
        plt.clf()
        del rho_fluct

        for rho_vals, resolution in zip([rho_lr_lvl_vals, rho_hr_lvl_vals], ['lowres', 'highres']):
            fig, ax = plt.subplots(subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,8))

            current_cmap = plt.get_cmap(name='hot_r').copy()
            current_cmap.set_bad(color='#bcb5bb')
            mapplot = plot_icon_tricolor(ax, lr_ds.clon_bnds.values, lr_ds.clat_bnds.values, rho_vals, cmap=current_cmap)

            ax.coastlines()
            ax.gridlines(draw_labels=True)
            fig.colorbar(mapplot, orientation='horizontal', aspect=30, pad=0.1, label=r'$\rho$ / kg m-3')
            fig.suptitle(f'{resolution} on vertical level {vlvl} ({z_fl_approx_km[vlvl]} km)')
            fig.savefig(os.path.join(fig_path, f'{resolution}_lvl{vlvl}.jpg'), dpi=150)
            plt.clf()
        
        del rho_lr_lvl_vals
        del rho_hr_lvl_vals
    del lr_ds
    del hr_ds

    # plt.show()
    # break

Date: 2013122300, hour: 0016
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013122300, hour: 0034
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013122500, hour: 0012
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70


/tmp/ipykernel_2545867/4104515226.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,8))


Vertical lvl: 74
Date: 2013122500, hour: 0028
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013122600, hour: 0016
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013122600, hour: 0019
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013122700, hour: 0014
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74
Date: 2013123000, hour: 0002
Vertical lvl: 0
Vertical lvl: 40
Vertical lvl: 70
Vertical lvl: 74


<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

...

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

# Comparison with/without neglection of horizontal fluctuations of rho

In [3]:
from glob import glob
import numpy.random as npr

files_old = np.array(sorted(glob(f'{base_path_project}/heuer23_ml_convection_parameterization/ProcessedRhoFluctNeglect/2013120[1-9]00/ParamPrep/LowRes/R02B05/*.nc')))
files_new = np.array(sorted(glob(f'{base_path_project}/heuer23_ml_convection_parameterization/Processed/2013120[1-9]00/ParamPrep/LowRes/R02B05/*.nc')))

In [4]:
print(len(files_old))
print(len(files_new))

333
333


In [32]:
rnd_idx = npr.choice(len(files_old), 2)
files_old_sample = files_old[rnd_idx]
files_new_sample = files_new[rnd_idx]

In [34]:
files_old_sample = [f'{base_path_project}heuer23_ml_convection_parameterization/ProcessedRhoFluctNeglect/2013120600/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120600_fg_DOM01_ML_0003_conv_out_R02B05_m2degr.nc', f'{base_path_project}heuer23_ml_convection_parameterization/ProcessedRhoFluctNeglect/2013120400/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120400_fg_DOM01_ML_0013_conv_out_R02B05_m2degr.nc']
files_new_sample = [f'{base_path_project}/heuer23_ml_convection_parameterization/Processed/2013120600/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120600_fg_DOM01_ML_0003_conv_out_R02B05_m2degr.nc', f'{base_path_project}/heuer23_ml_convection_parameterization/Processed/2013120400/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120400_fg_DOM01_ML_0013_conv_out_R02B05_m2degr.nc']
print(files_old_sample)
print(files_new_sample)

['/work/bd1179/b309215/heuer23_ml_convection_parameterization/ProcessedRhoFluctNeglect/2013120600/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120600_fg_DOM01_ML_0003_conv_out_R02B05_m2degr.nc', '/work/bd1179/b309215/heuer23_ml_convection_parameterization/ProcessedRhoFluctNeglect/2013120400/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120400_fg_DOM01_ML_0013_conv_out_R02B05_m2degr.nc']
['/work/bd1179/b309215/heuer23_ml_convection_parameterization/Processed/2013120600/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120600_fg_DOM01_ML_0003_conv_out_R02B05_m2degr.nc', '/work/bd1179/b309215/heuer23_ml_convection_parameterization/Processed/2013120400/ParamPrep/LowRes/R02B05/dei4_NARVALI_2013120400_fg_DOM01_ML_0013_conv_out_R02B05_m2degr.nc']


In [39]:
# xr.open_dataset(files_new_sample[0])

In [40]:
vertical_layers = [0, 10, 40, 70, 74]
variables = ['wu','wv']
# variables = ['wh']

for f_old, f_new in zip(files_old_sample, files_new_sample):
    date = re.search('_(201\d\d\d\d\d00)_', f_new).group(1)
    hr = re.search('_(00\d\d)_', f_new).group(1)
    ds_old = xr.open_dataset(f_old).dropna('cell')
    ds_new = xr.open_dataset(f_new).dropna('cell')
    for var in variables:
        # var_old = ds_old[var].values
        var_old = ds_new[var].values
        rhovar_new = ds_new['rho'+var].values
        rho_new = ds_new.rho.values
        var_new = rhovar_new/rho_new

        var_diff = var_new - var_old
        for vlvl in vertical_layers:
            fig, (ax2, ax) = plt.subplots(2,1,subplot_kw=dict(projection=cartopy.crs.PlateCarree()), figsize=(16,16))
            mapplot = plot_icon_tricolor(ax, ds_new.clon_bnds.values, ds_new.clat_bnds.values, var_diff.squeeze()[vlvl], cmap='bwr', norm = colors.TwoSlopeNorm(0))
            mapplot2 = plot_icon_tricolor(ax2, ds_new.clon_bnds.values, ds_new.clat_bnds.values, var_old.squeeze()[vlvl], cmap='bwr', norm = colors.TwoSlopeNorm(0))
            ax.coastlines()
            ax2.coastlines()
            ax.gridlines(draw_labels=True)
            ax2.gridlines(draw_labels=True)
            fig.colorbar(mapplot, ax=ax, orientation='horizontal', aspect=30, pad=0.1, label=rf'$\frac{{\overline{{{"rho"+var}}}}}{{\overline{{rho}}}} - \overline{{{var}}}$ / m2 s-2')
            # fig.colorbar(mapplot, ax=ax, orientation='horizontal', aspect=30, pad=0.1, label=rf'$\frac{{\overline{{{"rho"+var}}}}}{{\overline{{rho}}}} - \overline{{{var}}}$ / m s-1 J kg-1')
            fig.colorbar(mapplot2, ax=ax2, orientation='horizontal', aspect=30, pad=0.1, label=rf'$\overline{{{var}}}$ / m2 s-2')
            # fig.colorbar(mapplot2, ax=ax2, orientation='horizontal', aspect=30, pad=0.1, label=rf'$\overline{{{var}}}$ / m s-1 J kg-1')
            # fig.colorbar(mapplot, orientation='horizontal', aspect=30, pad=0.1, label=r'$(\frac{\overline{rhovar}}{\overline{rho}} - \overline{var}) / \overline{var}$')
            fig.suptitle(f'{date[:-2]} - {hr}h - lvl: {vlvl+1} - var: {var}')
            abs_values_lvl = np.abs(var_diff.squeeze()[vlvl] / var_old.squeeze()[vlvl])
            ax.set_title(f'Average absolute percentual difference: {np.mean(abs_values_lvl)*100:.2f}%, Median absolute percentual difference: {np.median(abs_values_lvl)*100:.2f}%')
            fig.savefig(f'Figures/rho_fluct_Y_influence/{date[:-2]}_{hr}h_lvl-{vlvl+1}_var{var}.jpg', dpi=150)
            plt.clf()
            # plt.show()

    # plt.show()

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>

<Figure size 1600x1600 with 0 Axes>